# 如何创建自定义提示模板

假设我们希望LLM能生成给定函数名称的英语语言解释。为了实现这个任务，我们将创建一个自定义提示模板，以函数名称作为输入，并格式化提示模板以提供函数的源代码。

## 为什么需要自定义提示模板？

LangChain提供了一套默认提示模板，可用于生成各种任务的提示。但是，有些情况下默认提示模板可能不符合您的需求。例如，您可能想要创建一个带有特定动态指令的提示模板以用于您的语言模型。在这种情况下，您可以创建自定义提示模板。

查看当前一组默认提示模板[在这里](../getting_started.md)。

## 创建自定义提示模板

实质上有两种不同的提示模板可用 - 字符串提示模板和聊天提示模板。字符串提示模板提供了一个简单的字符串格式提示，而聊天提示模板生成更结构化的提示以与聊天API一起使用。

在本指南中，我们将使用字符串提示模板创建自定义提示模板。

要创建自定义字符串提示模板，有两个要求：
1.它具有一个input_variables属性，公开提示模板期望的输入变量。
2.它公开一个format方法，该方法获取与期望的input_variables相对应的关键字参数，并返回格式化的提示。

我们将创建一个自定义提示模板，以函数名称作为输入，并格式化提示以提供函数的源代码。为了实现这个目标，让我们首先创建一个函数，该函数将返回给定名称函数的源代码。

In [2]:
import inspect

def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

接下来，我们将创建一个自定义提示模板，它以函数名称作为输入，并格式化提示模板以提供函数的源代码。


In [5]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator


class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """ A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function. """

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """ Validate that the input variables are correct. """
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = f"""
        Given the function name and source code, generate an English language explanation of the function.
        Function Name: {kwargs["function_name"].__name__}
        Source Code:
        {source_code}
        Explanation:
        """
        return prompt
    
    def _prompt_type(self):
        return "function-explainer"

## 使用自定义提示模板

既然我们已经创建了自定义提示模板，我们可以使用它来生成我们的任务的提示。

In [6]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)


        Given the function name and source code, generate an English language explanation of the function.
        Function Name: get_source_code
        Source Code:
        def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

        Explanation:
        
